In [1]:
from glob import glob

In [18]:
tag_paths = glob("KGRec-dataset/KGRec-music/tags/*.txt")
desc_paths = glob("KGRec-dataset/KGRec-music/descriptions/*.txt")

In [20]:
print("Tags:", len(tag_paths))
print("Descriptions:", len(desc_paths))

Tags: 8239
Descriptions: 8640


In [21]:
song_ids = [f"{i}" for i in range(1, 8640)]

In [14]:
all_tags = []

In [15]:
for tag in tag_paths:
    with open(tag, 'r') as f:
        lines = f.readlines()
        if len(lines) == 1:
            #print(lines[0].split(" "))
            all_tags.extend(lines[0].split(" "))

In [17]:
len(set(all_tags))

50956

In [22]:
songs_metadata = []